In [1]:
import os
import re
import arrow
import pickle

In [2]:
def format_time(time):
    time = time.split(":")
    time = [t.zfill(2) for t in time]
    time = ":".join(time)
    
    if not "00" <= time[:2] <= "23":
        hh = int(time[:2]) - 24
        hh = str(hh).zfill(2)
        time = list(time)
        time[:2] = hh
        time = "".join(time)
        
    return time

In [3]:
def is_consecutive(epoch1, epoch2):
    name1 = epoch1.name
    name2 = epoch2.name
    end1 = format_time(epoch1.end)
    start2 = format_time(epoch2.start)
    
    name_diff = int(epoch1.name[-6:-4]) - int(epoch2.name[-6:-4])
    time_diff = arrow.get(start2, 'HH:mm:ss') - arrow.get(end1, 'HH:mm:ss')
    
    if name1[3:5] == name2[3:5] and abs(name_diff) == 1 and time_diff.seconds < 300:
        return True
    
    return False

In [4]:
class Epoch:
    def __init__(self):
        self.name = ''
        self.start = 0
        self.end = 0
        self.number_of_seizures = 0
        self.seizures_starts = []
        self.seizures_ends = []
        self.duration = 0
        
    def __str__(self):
        return f"Epoch:\nname: {self.name}\nstart: {self.start}\nend: {self.end}\nnumber_of_seizures: {self.number_of_seizures}\nseizures_starts: {self.seizures_starts}\nseizures_ends: {self.seizures_ends}\nduration: {self.duration}\n"

In [5]:
class Seizure:
    def __init__(self):
        self.needed_epochs = []
        self.onset = 0
        
    def __str__(self):
        return f"needed_epochs: {self.needed_epochs}\nonset: {self.onset}\n"

In [6]:
epochs = []
patient_numbers = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23"]

In [7]:
for pt_num in patient_numbers:
    
    with open(f"/home/npe/Documents/Seizure-Prediction/chb-mit-scalp-eeg-database-1.0.0/chb{pt_num}/chb{pt_num}-summary.txt", 'r') as f:
        summary = f.read()
        
    summary = summary.split('\n')
    
    i = 0
    
    while True:
        if i >= len(summary):
            break
            
        if "File Name" in summary[i]:
            epochs.append(Epoch())
            file_name = summary[i].replace("File Name: ", '')
            epochs[-1].name = file_name
        
        if "File Start Time" in summary[i]:
            start = summary[i].replace("File Start Time: ", '')
            epochs[-1].start = start
            
        if "File End Time" in summary[i]:
            end = summary[i].replace("File End Time: ", '')
            epochs[-1].end = end
            
        if "Number of Seizures in File" in summary[i]:
            number_of_seizures = int(re.search('[0-9]+', summary[i]).group())
            epochs[-1].number_of_seizures = number_of_seizures
            
            for _ in range(number_of_seizures):
                seizures_start = int(re.search('[0-9]+', summary[i+1][summary[i+1].index(":") + 1:]).group())
                epochs[-1].seizures_starts.append(seizures_start)
                i += 1
                
                seizures_end = int(re.search('[0-9]+', summary[i+1][summary[i+1].index(":") + 1:]).group())
                epochs[-1].seizures_ends.append(seizures_end)
                i += 1
            
            start_time = epochs[-1].start
            end_time = epochs[-1].end
            
            start_time = format_time(start_time)
            end_time = format_time(end_time)
                        
            duration = arrow.get(end_time, 'HH:mm:ss') - arrow.get(start_time, 'HH:mm:ss')
            
            epochs[-1].duration = duration.seconds
                
            
        i += 1

In [8]:
needing_dur = 300 * 60
seizures = []

In [9]:
i = 0
number_of_epochs = len(epochs)

while i < number_of_epochs:
    for count in range(epochs[i].number_of_seizures):
        seizure = Seizure()

        onset = epochs[i].seizures_starts[count]
        seizure.onset = onset
        
        if count == 0 :
            duration = onset
            seizure.needed_epochs.append(epochs[i].name)
            j = i - 1 
            
            while duration < needing_dur:
                seizure.needed_epochs.append(epochs[j].name)
                
                if epochs[j].number_of_seizures == 0:
                    if is_consecutive(epochs[j], epochs[j+1]):
                        duration += epochs[j].duration
                    else:
                        break
                        
                else:
                    duration += epochs[j].duration - epochs[j].seizures_ends[-1] 
                    
                    if duration < needing_dur:
                        break
                        
                j -= 1
                
            if duration >= needing_dur:
                seizures.append(seizure)
            
                
        else:
            seizure.needed_epochs.append(epochs[i].name)
            
            if epochs[i].seizures_starts[count] - epochs[i].seizures_ends[count-1] >= needing_dur:
                seizures.append(seizure)
            
    
    i += 1
    

In [10]:
for s in seizures:
    print(s)

needed_epochs: ['chb01_15.edf', 'chb01_14.edf', 'chb01_13.edf', 'chb01_12.edf', 'chb01_11.edf', 'chb01_10.edf']
onset: 1732

needed_epochs: ['chb01_26.edf', 'chb01_25.edf', 'chb01_24.edf', 'chb01_23.edf', 'chb01_22.edf', 'chb01_21.edf']
onset: 1862

needed_epochs: ['chb04_05.edf', 'chb04_04.edf']
onset: 7804

needed_epochs: ['chb04_08.edf', 'chb04_07.edf', 'chb04_06.edf']
onset: 6446

needed_epochs: ['chb05_06.edf', 'chb05_05.edf', 'chb05_04.edf', 'chb05_03.edf', 'chb05_02.edf', 'chb05_01.edf']
onset: 417

needed_epochs: ['chb05_13.edf', 'chb05_12.edf', 'chb05_11.edf', 'chb05_10.edf', 'chb05_09.edf', 'chb05_08.edf']
onset: 1086

needed_epochs: ['chb06_04.edf', 'chb06_03.edf', 'chb06_02.edf']
onset: 327

needed_epochs: ['chb06_09.edf', 'chb06_08.edf']
onset: 12500

needed_epochs: ['chb06_13.edf', 'chb06_12.edf', 'chb06_10.edf']
onset: 506

needed_epochs: ['chb06_18.edf', 'chb06_17.edf']
onset: 7799

needed_epochs: ['chb07_12.edf', 'chb07_11.edf']
onset: 4920

needed_epochs: ['chb07_19.e

In [11]:
len(seizures)

23

In [12]:
with open("Seizures Data.pkl", 'wb') as f:
    pickle.dump(seizures, f)